In this notebook, you can find the code used to save the detections carried out by the models in the Pascal VOC format. They are later loaded into RoboFlow for manual adjustment.

In [ ]:
import os
import cv2
import pandas as pd
from PIL import Image

In [ ]:
xml_template_start = '''<annotation>
<folder/>
<filename>{}</filename>
<path>{}</path>
<size>
<width>1280</width>
<height>720</height>
<depth>3</depth>
</size>
<segmented>0</segmented>
'''

xml_object = '''<object>
<name>{}</name>
<pose>Unspecified</pose>
<truncated>0</truncated>
<difficult>0</difficult>
<occluded>0</occluded>
<bndbox>
<xmin>{}</xmin>
<xmax>{}</xmax>
<ymin>{}</ymin>
<ymax>{}</ymax>
</bndbox>
</object>
'''

xml_template_end = '</annotation>'

In [ ]:
def convert_string_to_list(string, convert_to_int=False, delimiter=","):
    converted_list = string.strip(')(][ ').replace("'", "").split(delimiter)
    
    if convert_to_int:
        converted_list = [int(element.strip()) for element in converted_list if element != ""]
    
    return converted_list

In [ ]:
def load_history_file(history_path):
    
    history_filename = 'history.csv'
    history = pd.read_csv(history_path+history_filename)
    
    # backup history file
    os.system(f'cp {history_path+history_filename} {history_path}history_bkp.csv')
    
    return history

In [ ]:
webcam_no = 2

webcam_path = f'/media/lucaszampar/BACKUP/webcam0{webcam_no}/' 

history_path = webcam_path + 'history/' 
extracted_images_path = webcam_path + 'extracted_images/' 

In [ ]:
history = load_history_file(history_path)

In [ ]:
#folders = ['bem_te_vi/', 'canario_do_amazonas/', 'sanhaco_da_amazonia/', 'sanhaco_do_coqueiro/', 'rolinha/']
#folders = ['chupim/']

In [ ]:
for species_folder in folders:    
    
    image_files = os.listdir(extracted_images_path + species_folder)
    
    for image_filename in image_files:

        xml = ''
        annotation_filename = image_filename.replace('.jpg', '.xml')
        image_filname_parts = image_filename.split('_')

        video_filename = '_'.join(image_filname_parts[0:3])+'.mp4'
        frame_pos = image_filname_parts[3]

        detection = history.query(f" filename == '{video_filename}' and frame_pos == {frame_pos}")

        labels = convert_string_to_list(detection['labels'].iloc[0]) 
        bboxes = detection['bboxes'].iloc[0].strip('][').split('), ')
        bboxes = [convert_string_to_list(bbox, convert_to_int=True) for bbox in bboxes]

        xml += xml_template_start.format(image_filename, image_filename)

        for label, bbox in zip(labels, bboxes):

            xmin, ymin, xmax, ymax = [int(point) for point in bbox ]
            xml += xml_object.format(label, xmin, xmax, ymin,  ymax)

        xml += xml_template_end

        with open(f'{extracted_images_path+species_folder}{annotation_filename}', 'w') as annotation_xml: 
            annotation_xml.write(xml)

        

